In [2]:
# REGRESSION
# 1. Split data: training set --> 70%, validation set --> 15%, and 15% testing set (Don’t use it while tunning the model parameters).
# 2a. Apply linear, lasso and ridge regression to the data to predict the median house value Implement from scratch using matrix operations. 
# 2b. Then Calculate weights using normal equation: w = (X^T X)^(-1) X^T y) 
# 2c. Apply Gradient descent as an alternative optimization approach
# 2c. Comment on the two results. 
# 2d. Apply L2 Regularization (Ridge Regression) and L1 Regularization (Lasso Regression) and try different regularization parameters 
# 2e. Plot validation error vs. regularization parameter
# 3. Re-apply step 2 again using Scikit- Learn and compare both results.
# 4. Report Mean Square Error and Mean Absolute Errors for all your models.
# 5. Add your comments on the results and compare between the models.

## 1. Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import SGDRegressor


## 2. Loading the Dataset

In [14]:
# Read the CSV file into a DataFrame
df = pd.read_csv('California_Houses.csv')

# Display the first few rows of the DataFrame
print(df.head())

   Median_House_Value  Median_Income  Median_Age  Tot_Rooms  Tot_Bedrooms  \
0            452600.0         8.3252          41        880           129   
1            358500.0         8.3014          21       7099          1106   
2            352100.0         7.2574          52       1467           190   
3            341300.0         5.6431          52       1274           235   
4            342200.0         3.8462          52       1627           280   

   Population  Households  Latitude  Longitude  Distance_to_coast  \
0         322         126     37.88    -122.23        9263.040773   
1        2401        1138     37.86    -122.22       10225.733072   
2         496         177     37.85    -122.24        8259.085109   
3         558         219     37.85    -122.25        7768.086571   
4         565         259     37.85    -122.25        7768.086571   

   Distance_to_LA  Distance_to_SanDiego  Distance_to_SanJose  \
0   556529.158342         735501.806984         67432.5170

## 3. Splitting the Dataset 

In [15]:
# X: All columns/features except the first one ("Median House Value")
X = df.iloc[:, 1:]

# Y: First Feature / Target ("Median House Value")
y = df.iloc[:, 0]

# First Split: 70% of data for training
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

# Second split: 50% of temp for validation, 50% for test (15% each)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


## 4. Feature Scaling (Normalisation)

In [16]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)


## 5. Without Sklearn

### a. Linear Regression

In [17]:
X_train_array = np.asarray(X_train_scaled)
X_val_array = np.asarray(X_val_scaled)

y_train_array = np.asarray(y_train)
y_val_array = np.asarray(y_val)

X_train_array = np.c_[np.ones(X_train_array.shape[0]), X_train_array]
X_val_array = np.c_[np.ones(X_val_array.shape[0]), X_val_array]

def linear_regression(X, y):
    weight = np.linalg.inv(X.T @ X) @ X.T @ y
    return weight

# Train the model
w = linear_regression(X_train_array, y_train_array)

# Make predictions
def predict(X, weight):
    return X @ weight

y_pred_linear = predict(X_val_array, w)

# Calculate Mean Squared Error 
mse_linear = np.mean((y_pred_linear - y_val_array)**2)
print(f"Linear Regression MSE: {mse_linear:.3f}")

# Calculate Mean Absolute Error
mae_linear = np.mean(abs(y_val_array - y_pred_linear))
print(f"Linear Regression MAE: {mae_linear:.3f}")

Linear Regression MSE: 4907211997.375
Linear Regression MAE: 50790.060


### b. Lasso Regression (L1)

In [37]:
def lasso_regression(X, y, alpha, learning_rate=0.01, n_iterations=1000):
    m = len(y)
    X_b = np.c_[np.ones((m, 1)), X]  # Add bias term (intercept)
    theta = np.random.randn(X_b.shape[1])  # Initialize weights
    
    for iteration in range(n_iterations):
        # Compute gradients (MSE part)
        gradients = (2/m) * X_b.T.dot(X_b.dot(theta) - y)
        
        # Apply L1 regularization (subgradient for non-differentiable L1 penalty)
        gradients[1:] += alpha * np.sign(theta[1:])  # Exclude bias term
        
        # Update weights
        theta -= learning_rate * gradients
    
    return theta

# Train model
w_lasso = lasso_regression(X_train_array, y_train_array, alpha=0.01, learning_rate=0.01, n_iterations=1000)

# Compute predictions
X_b_train = np.c_[np.ones((len(X_train_array), 1)), X_train_array]
y_pred_lasso = X_b_train.dot(w_lasso)

# Compute MSE
mse_lasso = np.mean((y_train_array - y_pred_lasso) ** 2) + 0.01 * np.sum(np.abs(w_lasso[1:]))
print(f"Lasso Regression MSE: {mse_lasso:.3f}")

# MAE 
mae_lasso = np.mean(np.abs(y_train_array - y_pred_lasso)) 
print(f"Lasso Regression MAE: {mae_lasso:.3f}")


Lasso Regression MSE: 4820463143.295
Lasso Regression MAE: 50839.516


### c. Ridge Regression (L2)

In [ ]:
def ridge_regression(X, y, alpha, learning_rate=0.01, n_iterations=1000):
    m = len(y)
    X_b = np.c_[np.ones((m, 1)), X]
    theta = np.random.randn(X_b.shape[1])
    for iteration in range(n_iterations):
        gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
        # Apply regularization only to theta[1:] (excluding bias)
        gradients[1:] += 2 * alpha * theta[1:]
        theta -= learning_rate * gradients
    return theta

# Train model
w_ridge = ridge_regression(X_train_array, y_train_array, alpha=0.01, learning_rate=0.01, n_iterations=1000)

# Compute predictions
X_b_train = np.c_[np.ones((len(X_train_array), 1)), X_train_array]
y_pred_ridge = X_b_train.dot(w_ridge)

# Compute MSE with ridge penalty
mse_ridge = np.mean((y_train_array - y_pred_ridge) ** 2) + 0.01 * np.sum(w_ridge[1:] ** 2)
print(f"Ridge Regression MSE: {mse_ridge:.3f}")

# Compute MAE without regularization penalty (normally no penalty added to MAE)
mae_ridge = np.mean(np.abs(y_train_array - y_pred_ridge))
print(f"Ridge Regression MAE: {mae_ridge:.3f}")

Ridge Regression MSE: 5023174659.308
Ridge Regression MAE: 50898.646


### d. Gradient Descent 

In [20]:
def gradient_descent(X, y, learning_rate=0.001, n_iterations=1000):
    N = len(y)  # Number of training examples
    w = np.zeros(X.shape[1])
    cost_history = []

    for _ in range(n_iterations):
        # 1. Compute predictions (y = Xw)
        predictions = X.dot(w)
        
        # 2. Compute errors (y - t)
        errors = predictions - y
        
        # 3. Compute gradient
        gradient = (1 / N) * X.T.dot(errors)
        
        # 4. Update weights
        w = w - learning_rate * gradient
    
        # 5. Calculating Cost Function
        J = 0.5*np.mean(errors**2)
        cost_history.append(J)
        
        print(f"Iteration {_}: Cost = {J}")
        
    return w, cost_history

# Run gradient descent
theta, cost_history = gradient_descent(X_train_array, y_train_array, learning_rate=0.001, n_iterations=1000)

Iteration 0: Cost = 28107242683.440372
Iteration 1: Cost = 28053963846.44588
Iteration 2: Cost = 28000800675.31327
Iteration 3: Cost = 27947752886.33103
Iteration 4: Cost = 27894820196.793674
Iteration 5: Cost = 27842002324.995293
Iteration 6: Cost = 27789298990.223167
Iteration 7: Cost = 27736709912.75142
Iteration 8: Cost = 27684234813.83474
Iteration 9: Cost = 27631873415.7022
Iteration 10: Cost = 27579625441.551052
Iteration 11: Cost = 27527490615.54067
Iteration 12: Cost = 27475468662.786457
Iteration 13: Cost = 27423559309.35393
Iteration 14: Cost = 27371762282.252724
Iteration 15: Cost = 27320077309.43076
Iteration 16: Cost = 27268504119.768402
Iteration 17: Cost = 27217042443.07271
Iteration 18: Cost = 27165692010.071705
Iteration 19: Cost = 27114452552.408745
Iteration 20: Cost = 27063323802.63687
Iteration 21: Cost = 27012305494.213306
Iteration 22: Cost = 26961397361.4939
Iteration 23: Cost = 26910599139.72773
Iteration 24: Cost = 26859910565.051643
Iteration 25: Cost = 2680

## 6. With Sklearn

### a. Regression Models

In [21]:
# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Initialize models
linear_model = LinearRegression()
lasso_model = Lasso(alpha=0.1, max_iter=10000)
ridge_model = Ridge(alpha=0.1, max_iter=10000)

# Fit the models
linear_model.fit(X_train_scaled, y_train)
lasso_model.fit(X_train_scaled, y_train)
ridge_model.fit(X_train_scaled, y_train)

# Make predictions
y_pred_linear_sk = linear_model.predict(X_val_scaled)
y_pred_lasso_sk = lasso_model.predict(X_val_scaled)
y_pred_ridge_sk = ridge_model.predict(X_val_scaled)

# Calculate MSE and MAE
mse_linear_sk = mean_squared_error(y_val, y_pred_linear_sk)
mae_linear_sk = mean_absolute_error(y_val, y_pred_linear_sk)

mse_lasso_sk = mean_squared_error(y_val, y_pred_lasso_sk)
mae_lasso_sk = mean_absolute_error(y_val, y_pred_lasso_sk)

mse_ridge_sk = mean_squared_error(y_val, y_pred_ridge_sk)
mae_ridge_sk = mean_absolute_error(y_val, y_pred_ridge_sk)

# Print the results
print(f"Linear Regression - MSE: {mse_linear_sk:.3f}, MAE: {mae_linear_sk:.3f}")
print(f"Lasso Regression - MSE: {mse_lasso_sk:.3f}, MAE: {mae_lasso_sk:.3f}")
print(f"Ridge Regression - MSE: {mse_ridge_sk:.3f}, MAE: {mae_ridge_sk:.3f}")

Linear Regression - MSE: 4907211997.375, MAE: 50790.060
Lasso Regression - MSE: 4907213520.759, MAE: 50790.132
Ridge Regression - MSE: 4907216086.179, MAE: 50790.421


### b. Gradient Descent using Sklearn

In [23]:
# Initialize Mini-Batch GD models with better defaults
mbgd_linear = SGDRegressor(
    loss = 'squared_error', # Loss Function
    penalty = None, # Controls Regularization (prevents overfitting)
    max_iter=5000, # Max no. of epochs
    eta0=0.01, # Learning Rate 
)

mbgd_lasso = SGDRegressor(
    loss='squared_error', # Loss Function
    penalty='l1', # Lasso Regularization
    alpha=0.0001,  # Regularization Strength
    max_iter=5000, # Max no. of epochs
    eta0=0.01, # Learning Rate
)

mbgd_ridge = SGDRegressor(
    loss='squared_error', # Loss Function
    penalty='l2', # Ridge Regularization
    alpha=0.01, # Regularization Strength
    max_iter=5000, # Max no. of epochs
    eta0=0.01, # Learning Rate
)